# Post-procesamiento

Este notebook describe los métodos de post-procesamiento que fueron aplicados sobre los resultados de las predicciones del modelo de U-Net.

## Poligonizado

La siguiente función aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina aplica `gdal_polygonize.py` a cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1).

In [3]:
!pip install -e /home/ro/satproc/.


Obtaining file:///home/ro/satproc
  Attempting uninstall: satproc
    Found existing installation: satproc 0.0.post0.dev99+g3977d20.dirty
    Uninstalling satproc-0.0.post0.dev99+g3977d20.dirty:
      Successfully uninstalled satproc-0.0.post0.dev99+g3977d20.dirty
  Running setup.py develop for satproc
You should consider upgrading via the '/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
from satproc.postprocess import polygonize

In [2]:
!mkdir /home/ro/basurales/data/data_results/filtered_up05/

In [3]:
filter_by_max_prob(input_dir='/home/ro/basurales/data/data_results/6D/100_100',
                   output_dir='/home/ro/basur147853
                   ales/data/data_results/filtered_up05/',
                   threshold=0.5)

100%|██████████| 178235/178235 [03:33<00:00, 833.13it/s]


In [ ]:
input_path = '/home/ro/basurales/data/data_results/filtered_up05/'
output_path = '/home/ro/basurales/data/data_results/basurales_100x100_thr05.gpkg'


polygonize(threshold=0.5,
           input_dir=input_path,
           output=output_path)

 97%|█████████▋| 55752/57633 [2:31:01<05:46,  5.43it/s]  

## Filtrar por área mínima

Como último paso, filtramos aquellos polígonos de area inferior a 300 m², dado que consideramos que son falsos positivos. Para esto utilizamos `ogr2ogr` y una consulta SQL. El resultado se guarda en `data/results/predict_300.gpkg`

In [18]:
min_area = 35000
input_path = "/home/ro/basurales/data/data_results/basurales_100x100_thr05_utm.gpkg"
output_path = "/home/ro/basurales/data/data_results/basurales_100x100_thr05_up35000_utm.gpkg"

In [4]:
!ogrinfo -al $input_path | head -n25

INFO: Open of `/home/ro/basurales/data/data_results/basurales_100x100_thr05_utm.gpkg'
      using driver `GPKG' successful.

Layer name: basurales_100x100_thr05_utm
Geometry: Polygon
Feature Count: 111253
Extent: (455592.599528, 5473248.203130) - (1070896.286680, 6318803.329936)
Layer SRS WKT:
PROJCRS["WGS 84 / UTM zone 20S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 20S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-63,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],


In [19]:
!ogr2ogr \
    -t_srs EPSG:32720 \
    -f "GPKG" \
    -sql "SELECT * FROM basurales_100x100_thr05_utm m WHERE ST_Area(geom) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $output_path \
    $input_path